In [1]:
import pandas as pd
import json

In [2]:

ruta='0pisos_limpios.csv'
dfknn = pd.read_csv(ruta,sep=',')
dfknn.columns

Index(['uv', 'estado', 'precio', 'long', 'lat', 'provincia', 'ciudad',
       'distrito', 'planta', 'habitaciones', 'banos', 'metros',
       'fecha_construccion', 'intext', 'portero', 'ascensor',
       'parque_infantil', 'terraza', 'trastero', 'cuarto_de_basura',
       'zonas_comunes', 'piscina', 'garaje', 'calefaccion', 'jardin',
       'aire_acondicionado', 'cusec', 'precio_area', 'reforma',
       'precio_reformado', 'ns', 'eo'],
      dtype='object')

In [3]:
with open('pesos.json') as archivo:
            opciones = json.load(archivo)

In [4]:
filtros=opciones['filtros']


In [5]:
dfknn.columns

Index(['uv', 'estado', 'precio', 'long', 'lat', 'provincia', 'ciudad',
       'distrito', 'planta', 'habitaciones', 'banos', 'metros',
       'fecha_construccion', 'intext', 'portero', 'ascensor',
       'parque_infantil', 'terraza', 'trastero', 'cuarto_de_basura',
       'zonas_comunes', 'piscina', 'garaje', 'calefaccion', 'jardin',
       'aire_acondicionado', 'cusec', 'precio_area', 'reforma',
       'precio_reformado', 'ns', 'eo'],
      dtype='object')

In [6]:
filtros=filtros[7:9]
filtros

['banos', 'portero']

In [7]:
def haversine(d, tar):
    tar = tar.loc[tar.index.repeat(len(d))]
    t=pd.DataFrame(tar)
    indice = d.index
    print(t)
    t.set_index(indice, inplace=True)

    lon1 = d.long
    lat1 = d.lat

    lon2 = t.long
    lat2 = t.lat
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    
    km = 6371* c
    return km

In [8]:
df = dfknn
target = df.iloc[1]
print(df[;,1])

df.drop(1, axis=0, inplace=True)

for f in filtros:
    index = df[ df[f] != target[f]  ].index
    df.drop(index, axis=0, inplace=True)
    df.drop(f,axis=1, inplace=True)

    #calcula geo y asceonsor

df['geo'] = haversine(df[['long','lat']], target[['long','lat']])





KeyError: 1

In [10]:
df

,uv,estado,precio,long,lat,provincia,ciudad,distrito,planta,habitaciones,...,garaje,calefaccion,jardin,aire_acondicionado,cusec,precio_area,reforma,precio_reformado,ns,eo
0,UV/2020/000003,1,160.00,40.4278,-3.63083,1,Madrid,San Blas,9,2,...,0,0,0,0,2.807915e+09,167909.090909,35,195.00,N,E
2,UV/2020/000006,1,0.00,40.4330,-3.63715,1,Madrid,Ciudad Lineal,2,2,...,0,0,0,0,2.807915e+09,136690.000000,35,35.00,S,NaN
3,UV/2020/000007,1,159.00,40.2950,-3.80339,1,Fuenlabrada,La Serna,9,3,...,0,0,0,0,2.805806e+09,NaN,35,194.00,S,E
4,UV/2020/000008,1,170.00,40.2846,-3.78862,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,45,215.00,S,E
5,UV/2020/000009,1,170.00,40.2839,-3.78976,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,35,205.00,N,E
6,UV/2020/000010,1,265.00,40.4107,-3.70752,1,Madrid,Centro,1,2,...,0,0,0,0,2.807901e+09,309714.285714,35,300.00,S,E
8,UV/2020/000013,1,260.00,40.4268,-3.66187,1,Madrid,Salamanca,2,2,...,0,0,0,0,2.807904e+09,306000.000000,35,295.00,N,E
9,UV/2020/000014,1,135.00,40.4101,-3.70373,1,Madrid,Centro,1,1,...,0,0,0,0,2.807901e+09,289513.636364,35,170.00,N,E
11,UV/2020/000016,1,120.00,40.3834,-3.70528,1,Madrid,Usera,3,2,...,0,0,0,0,2.807912e+09,127887.777778,35,155.00,S,E
13,UV/2020/000018,1,169.00,40.2919,-3.78604,1,Fuenlabrada,La Avanzada - La Cueva,9,3,...,0,0,0,0,2.805802e+09,NaN,45,214.00,S,E


KeyError: "None of [Index(['banos', 'portero'], dtype='object')] are in the [columns]"

In [11]:


        index = df[ df[f] != target[f]  ].index
        df.drop(index, axis=0, inplace=True)
        
        


KeyError: 'portero'

In [10]:
df

,uv,estado,precio,long,lat,provincia,ciudad,distrito,planta,habitaciones,...,garaje,calefaccion,jardin,aire_acondicionado,cusec,precio_area,reforma,precio_reformado,ns,eo
1,UV/2020/000004,1,125.0,40.1924,-3.66922,1,Valdemoro,Zona estación,1,2,...,0,0,0,0,2.816101e+09,NaN,35,160.0,S,E
2,UV/2020/000006,1,0.0,40.4330,-3.63715,1,Madrid,Ciudad Lineal,2,2,...,0,0,0,0,2.807915e+09,136690.000000,35,35.0,S,NaN
3,UV/2020/000007,1,159.0,40.2950,-3.80339,1,Fuenlabrada,La Serna,9,3,...,0,0,0,0,2.805806e+09,NaN,35,194.0,S,E
4,UV/2020/000008,1,170.0,40.2846,-3.78862,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,45,215.0,S,E
5,UV/2020/000009,1,170.0,40.2839,-3.78976,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,35,205.0,N,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,UV/2021/000378,1,225.0,41.3735,2.17075,2,Barcelona,Sants-Montjuïc,7,2,...,0,0,0,0,8.019030e+08,272568.421053,35,260.0,S,E
159,UV/2021/000428,1,125.0,40.3557,-3.69495,1,Madrid,Villaverde,4,3,...,0,0,0,0,2.807917e+09,149666.666667,45,170.0,S,NaN
161,UV/2021/000430,2,239.0,41.3745,2.12767,2,Barcelona,Sants-Montjuïc,2,2,...,0,0,0,0,8.019031e+08,236071.428571,35,274.0,S,E
162,UV/2021/000433,1,300.0,41.3809,2.17924,2,Barcelona,Ciutat Vella,0,4,...,0,0,0,0,8.019010e+08,477259.797980,55,355.0,S,NaN
